ДООБУЧЕНИЕ STABLE AUDIO AI

### 1. Подготовка данных   
- Описание: На данном этапе собираются и подготавливаются данные, которые будут использоваться для обучения модели.
   - Что включает:     
    - Сбор исходных данных: аудио файлы, аннотации, метаданные.
     - Очистка данных: удаление шума, лишней информации или ошибок.     
     - Разметка: создание соответствия между аудио и текстовыми транскрипциями.
     - Форматирование данных: преобразование аудио в нужный формат (в нашем случае mp3 -> wav) и создание необходимой структуры директорий.     
     - Разделение данных: разделение на обучающую, валидационную и тестовую выборки.
---
### 2. Загрузка предварительно обученной модели Stable Audio
   - Описание: Использование уже существующей модели, обученной на схожей задаче (pre-trained model).   
   - Что включает:
     - Загрузка предварительно обученной модели, которая предоставляет основу для дообучения.     
     - Проверка совместимости модели с текущими задачами и доступными данными.
     - Анализ параметров модели: размер, архитектура, функции потерь, методы оптимизации.   
     - Зачем нужно: Предобученная модель помогает ускорить обучение и повысить качество, поскольку она уже "знает" базовые аспекты задачи.
---
### 3. Настройка модели и компонентов
   - Описание: Этап адаптации модели к специфике задачи и среды обучения.   
   - Что включает:
     - Настройка гиперпараметров: скорость обучения, размер батча, количество эпох.     
     - Адаптация архитектуры модели: добавление новых слоев, изменение функции активации.
     - Настройка компонентов: выбор оптимизаторов (Adam), функций потерь (MSE).     
     - Подготовка среды обучения: выбор фреймворков (PyTorch), настройка GPU.
     - Добавление регуляризации или других методов для предотвращения переобучения.
---
### 4. Цикл дообучения   - Описание: Основной процесс обучения модели на предоставленных данных.
   - Что включает:     
     - Загрузка данных в модель в виде батчей.
     - Прохождение прямого распространения (forward pass) через модель.     
     - Вычисление функции потерь для оценки ошибки.
     - Обратное распространение (backpropagation) для обновления весов модели.     
     - Валидация на отложенной выборке для проверки качества обучения.
     - Повторение процесса в течение заданного количества эпох.   
     - Особенности:
     - Мониторинг метрик (точность, потери, время обучения).     
     - Возможность остановки обучения при достижении заданных условий (early stopping).
---
### 5. Сохранение модели
   - Описание: Этап фиксации текущего состояния модели после успешного обучения.   
   - Что включает:
     - Сохранение весов модели в файл.     
     - Сохранение структуры модели и гиперпараметров.
     - Версионирование: создание уникальных версий модели для управления изменениями.     
     - Оптимизация модели для дальнейшего использования: преобразование в формат подходящий для внедрения.
---
### 6. Проверка качества модели
   - Описание: Оценка обученной модели на основе ее работы на тестовых данных.   
   - Что включает:
     - Тестирование на данных, которые не участвовали в обучении.     
     - Расчет метрик качества: точность, полнота, F1-score, среднеквадратичная ошибка и др.
     - Анализ ошибок: выявление случаев, где модель дает некорректные результаты.     
     - Сравнение с базовыми метриками (baseline) или предыдущими версиями модели.
     - Генерация отчетов и визуализация результатов (например, графики точности/потерь, confusion matrix).

Requirements

In [ ]:
! pip install torch==2.0.1
! pip install torchaudio==2.0.2
! pip install torchvision==0.15.2
! pip install transformers>=4.41.0,<5.0.0
! pip install diffusers==0.17.0
! pip install flax==0.6.10
! pip install jax==0.4.27
! pip install jaxlib==0.4.27
! pip install sentence-transformers==2.2.2
! pip install huggingface_hub==0.23.0

Логин hugging face

In [ ]:
# Логин в Hugging Face (введите свой токен безопасно)
from huggingface_hub import notebook_login
notebook_login()

Необходимые импорты

In [ ]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torchaudio
import pandas as pd
import os
from diffusers import DiffusionPipeline, UNetConditionalModel
from transformers import AutoTokenizer, AutoModel
from tqdm.auto import tqdm

Подключение к Google диску

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Загрузка модели

In [ ]:
# Загрузка предварительно обученной модели Stable Audio
model_id = "stabilityai/stable-audio-1-0"
pipeline = DiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipeline.to("cuda")

Подготовка данных

In [ ]:
class AudioTextDataset(Dataset):
    def __init__(self, tsv_path, audio_dir, tokenizer, max_audio_length=16000, max_text_length=128):
        self.data = pd.read_csv(tsv_path, sep="\t")
        self.audio_dir = audio_dir
        self.tokenizer = tokenizer
        self.max_audio_length = max_audio_length
        self.max_text_length = max_text_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        text = row['sentence']
        audio_path = os.path.join(self.audio_dir, row['path'])

        # Загрузка аудио
        audio, sr = torchaudio.load(audio_path)

        # Преобразование в моно и ресемплинг до 16000 Гц
        if sr != 16000:
            resampler = torchaudio.transforms.Resample(sr, 16000)
            audio = resampler(audio)
        if audio.shape[0] > 1:
            audio = torch.mean(audio, dim=0, keepdim=True)

        # Обрезка или дополнение аудио до max_audio_length
        audio = self._pad_or_trim(audio.squeeze(0), self.max_audio_length)

        # Токенизация текста
        tokens = self.tokenizer(
            text,
            padding="max_length",
            truncation=True,
            max_length=self.max_text_length,
            return_tensors="pt",
        )

        return {
            "audio": audio,
            "input_ids": tokens.input_ids.squeeze(0),
            "attention_mask": tokens.attention_mask.squeeze(0),
        }

    def _pad_or_trim(self, audio, length):
        if len(audio) > length:
            audio = audio[:length]
        else:
            audio = torch.nn.functional.pad(audio, (0, length - len(audio)))
        return audio


Настраиваем токенизатор и пути к файлам

In [ ]:
# Используем предобученный токенизатор
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")

# Путь к вашим данным
tsv_path = "/path/to/your/validated.tsv"
audio_dir = "/path/to/your/clips"

# Создаем датасет и загрузчик
dataset = AudioTextDataset(tsv_path, audio_dir, tokenizer)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)


UNet & pipelines

In [ ]:
# Размораживаем только UNet для обучения
unet = pipeline.unet
unet.train()

# Замораживаем остальные части модели
pipeline.vae.eval()
pipeline.text_encoder.eval()

# Оптимизатор
optimizer = torch.optim.AdamW(unet.parameters(), lr=1e-5)


Настройка обучения (гиперпарамаетры)

In [ ]:
def train_loop(dataloader, unet, optimizer, epochs=3):
    for epoch in range(epochs):
        print(f"Epoch {epoch+1}/{epochs}")
        pbar = tqdm(dataloader)
        for batch in pbar:
            optimizer.zero_grad()

            # Получаем данные
            audio = batch['audio'].to("cuda", dtype=torch.float16)
            input_ids = batch['input_ids'].to("cuda")
            attention_mask = batch['attention_mask'].to("cuda")

            # Генерируем шум
            noise = torch.randn_like(audio)

            # Генерируем временные шаги
            timesteps = torch.randint(0, pipeline.scheduler.config.num_train_timesteps, (audio.shape[0],), device=audio.device).long()

            # Получаем эмбеддинги текста
            with torch.no_grad():
                encoder_hidden_states = pipeline.text_encoder(input_ids)[0]

            # Предсказываем шум
            noise_pred = unet(audio.unsqueeze(1), timesteps, encoder_hidden_states).sample.squeeze(1)

            # Вычисляем потери
            loss = nn.MSELoss()(noise_pred, noise)

            # Обратное распространение
            loss.backward()
            optimizer.step()

            pbar.set_postfix({"loss": loss.item()})


ЗАПУСК ОБУЧЕНИЯ

In [ ]:
train_loop(dataloader, unet, optimizer, epochs=3)


Сохранение обученной модели

In [ ]:
# Сохраняем дообученный UNet
unet.save_pretrained("./finetuned_unet")

# При желании можно загрузить дообученный UNet позже
# pipeline.unet = UNetConditionalModel.from_pretrained("./finetuned_unet")


Тестирование модели

In [ ]:
# Устанавливаем модель в режим оценки
unet.eval()

# Пример текстового запроса
prompt = "Пример текста для пения на русском языке"

# Токенизация текста
tokens = tokenizer(
    prompt,
    padding="max_length",
    truncation=True,
    max_length=128,
    return_tensors="pt",
)

input_ids = tokens.input_ids.to("cuda")
attention_mask = tokens.attention_mask.to("cuda")

# Генерируем аудио
with torch.no_grad():
    audio_output = pipeline(
        prompt=prompt,
        num_inference_steps=50,
        guidance_scale=7.5,
        generator=torch.manual_seed(0)
    ).audios[0]

# Сохраняем аудио
torchaudio.save("generated_audio.wav", torch.tensor(audio_output).unsqueeze(0), sample_rate=16000)
